In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('processed_data/train_data.csv')
pd.set_option('display.max_rows', 500)

In [10]:
#df.head(10)

In [9]:
X = df.drop(['Label','RunID'], axis=1)
y = df['Label']

# Remove classes with fewer than 2 instances
y = y[y.map(y.value_counts()) >= 2]
X = X.loc[y.index]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the classifiers and their hyperparameter grids
classifiers = {
    'RandomForest': (RandomForestClassifier(), {
        'n_estimators': [50, 100, 150, 200, 250],
        'max_depth': [None, 5, 10, 15, 20, 25],
        'min_samples_split': [2, 5, 10, 15, 20]
    }),
    'GradientBoosting': (GradientBoostingClassifier(), {
        'n_estimators': [50, 100, 150, 200, 250],
        'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2],
        'max_depth': [3, 5, 7, 9, 11]
    }),
    'SVC': (SVC(), {
        'C': [0.1, 0.5, 1, 5, 10],
        'gamma': ['scale', 'auto'],
        'kernel': ['rbf', 'linear', 'poly', 'sigmoid']
    }),
    'KNN': (KNeighborsClassifier(), {
        'n_neighbors': [3, 5, 7, 9, 11],
        'weights': ['uniform', 'distance']
    }),
    'DecisionTree': (DecisionTreeClassifier(), {
        'max_depth': [None, 5, 10, 15, 20],
        'min_samples_split': [2, 5, 10, 15, 20]
    }),
    'NaiveBayes': (GaussianNB(), {
        'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5]
    })
}


# Dictionary to store the best models and their results
best_models = {}
results = []

# Perform hyperparameter tuning and evaluation for each classifier
for clf_name, (clf, param_grid) in classifiers.items():
    grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
    grid_search.fit(X_train, y_train)
    
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, zero_division=0)
    
    best_models[clf_name] = best_model
    results.append({
        'Classifier': clf_name,
        'Best Params': grid_search.best_params_,
        'Accuracy': accuracy,
        'Classification Report': report
    })
    print(f"Classifier: {clf_name}")
    print(f"Best Params: {grid_search.best_params_}")
    print(f"Accuracy: {accuracy}")
    print("Classification Report:")
    print(report)
    print("\n" + "="*80 + "\n")

# Displaying the summary of results
results_df = pd.DataFrame(results)
print(results_df)

Classifier: RandomForest
Best Params: {'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 50}
Accuracy: 0.46987951807228917
Classification Report:
              precision    recall  f1-score   support

         4.0       0.00      0.00      0.00         2
         5.0       0.33      0.30      0.32        10
         6.0       0.00      0.00      0.00        11
         7.0       0.49      0.88      0.63        33
         8.0       0.50      0.26      0.34        23
         9.0       0.50      0.25      0.33         4

    accuracy                           0.47        83
   macro avg       0.30      0.28      0.27        83
weighted avg       0.40      0.47      0.40        83



Classifier: GradientBoosting
Best Params: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 50}
Accuracy: 0.42168674698795183
Classification Report:
              precision    recall  f1-score   support

         4.0       0.00      0.00      0.00         2
         5.0       0.20      0.10      